In [1]:
import numpy as np
import tensorflow as tf

data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))
    
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20], -1., -1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))
    
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20,3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)
    
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    
    tf.summary.scalar('cost', cost) #하나의 값: scalar, histogram, image, audio 등 다양한 값을 수집하는 함수를 기본으로 제공
#     각 가중치와 편향 등의 변화를 그래프로 살펴보고 싶다면 학습에 다음 코드를 넣고 진행하면 DISTRIBUTION, HISTOGRAMS 메뉴를 통해 확인 가능
#     tf.summary.histogram("Weights", W1)

In [3]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
# if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
#     saver.restore(sess, ckpt.model_checkpoint_path)
# else:
#     sess.run(tf.global_variables_initializer())
sess.run(tf.global_variables_initializer())

In [4]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [5]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    if step%10==0:
        print('step: %d' % sess.run(global_step),
              'cost: %.3f' % sess.run(cost, feed_dict={X:x_data, Y:y_data})
    
    summary = sess.run(merged, feed_dict={X:x_data, Y:y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

SyntaxError: invalid syntax (<ipython-input-5-ba7d8740e4bf>, line 8)

In [ ]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

In [ ]:
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('prediction: ', sess.run(prediction, feed_dict={X: x_data, Y:y_data}))
print('real value(target): ', sess.run(target, feed_dict={Y:y_data}))

is_correct=tf.equal(prediction, target)
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('accuracy: %.3f' % sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data}))

#결과 확인
#0: 기타, 1:포유류, 2:조류

log 디텍터리 생성됨
터미널에서 다음을 입력하여 웹서버 실행

tensorboard --logdir=./logs

내용을 확인하려면, 웹브라우저에서 다음 실행

http://localhost:6006/